# Rotation CIFAR Experiment

This experiment will use images from the **CIFAR-100** database (https://www.cs.toronto.edu/~kriz/cifar.html) and showcase the backward transfer efficiency of algorithms in the **Progressive Learning** project (https://github.com/neurodata/progressive-learning) as the images are rotated.

In [1]:
# Import the packages for experiment
import warnings
warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
import random
from skimage.transform import rotate
from scipy import ndimage
from skimage.util import img_as_ubyte
from joblib import Parallel, delayed
from sklearn.ensemble.forest import _generate_unsampled_indices
from sklearn.ensemble.forest import _generate_sample_indices
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from itertools import product
import keras
from keras import layers
from joblib import Parallel, delayed
from multiprocessing import Pool
import tensorflow as tf
from numba import cuda
import seaborn as sns

In [2]:
# Import the progressive learning packages
from proglearn.progressive_learner import ProgressiveLearner
from proglearn.network import LifelongClassificationNetwork
from proglearn.forest import LifelongClassificationForest
from proglearn.deciders import SimpleAverage
from proglearn.transformers import TreeClassificationTransformer, NeuralClassificationTransformer 
from proglearn.voters import TreeClassificationVoter, KNNClassificationVoter

# Create array to store errors
errors_array = []

TabError: inconsistent use of tabs and spaces in indentation (forest.py, line 93)

In [ ]:
# Randomized selection of training and testing subsets
def cross_val_data(data_x, data_y, total_cls=10):
    x = data_x.copy()
    y = data_y.copy()
    idx = [np.where(data_y == u)[0] for u in np.unique(data_y)]
    
    
    for i in range(total_cls):
        indx = idx[i]#np.roll(idx[i],(cv-1)*500)
        random.shuffle(indx)
        
        if i==0:
            train_x1 = x[indx[0:250],:]
            train_x2 = x[indx[250:500],:]
            train_y1 = y[indx[0:250]]
            train_y2 = y[indx[250:500]]
            
            test_x = x[indx[500:600],:]
            test_y = y[indx[500:600]]
        else:
            train_x1 = np.concatenate((train_x1, x[indx[0:250],:]), axis=0)
            train_x2 = np.concatenate((train_x2, x[indx[250:500],:]), axis=0)
            train_y1 = np.concatenate((train_y1, y[indx[0:250]]), axis=0)
            train_y2 = np.concatenate((train_y2, y[indx[250:500]]), axis=0)
            
            test_x = np.concatenate((test_x, x[indx[500:600],:]), axis=0)
            test_y = np.concatenate((test_y, y[indx[500:600]]), axis=0)
        
    return train_x1, train_y1, train_x2, train_y2, test_x, test_y 

# Algorithms

The progressive-learning repo contains two main algorithms, **Lifelong Learning Forests** (L2F) and **Lifelong Learning Network** (L2N), the difference being that L2F uses the Uncertainty Forest transformer while L2N uses deep neural networks. Both algorithms, unlike LwF, EWC, Online_EWC, and SI, have been shown to achieve both forward and backward knowledge transfer. Either algorithm can be chosen for the purpose of this experiment.

# Experiment

If the chosen algorithm is trained on both straight up-and-down CIFAR images and rotated CIFAR images, rather than just straight up-and-down CIFAR images, will it perform better (achieve a higher backward transfer efficiency) when tested on straight up-and-down CIFAR images? How does the angle at which training images are rotated affect these results?

At a rotation angle of 0 degrees, the rotated images simply provide additional straight up-and-down CIFAR training data, so the backward transfer efficiency at this angle show whether or not the chosen algorithm can even achieve backward knowledge transfer. As the angle of rotation increases, the rotated images become less and less similar to the original dataset, so the backward transfer efficiency should logically decrease, while still being above 1.

In [ ]:
# Chooses model to use as transformer
def choose_transformer(train_x1, test_x, test_y, tmp_data):
    
    # Deep Neural Networks model is used as transformer
    if model == "dnn":

        default_transformer_class = NeuralClassificationTransformer

        network = keras.Sequential()
        network.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=np.shape(train_x1)[1:]))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=32, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=64, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=128, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Conv2D(filters=254, kernel_size=(3, 3), strides = 2, padding = "same", activation='relu'))

        network.add(layers.Flatten())
        network.add(layers.BatchNormalization())
        network.add(layers.Dense(2000, activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Dense(2000, activation='relu'))
        network.add(layers.BatchNormalization())
        network.add(layers.Dense(units=10, activation = 'softmax'))

        default_transformer_kwargs = {"network" : network, 
                                      "euclidean_layer_idx" : -2,
                                      "num_classes" : 10,
                                      "optimizer" : keras.optimizers.Adam(3e-4)
                                     }

        default_voter_class = KNNClassificationVoter
        default_voter_kwargs = {"k" : int(np.log2(len(train_x1)))}

        default_decider_class = SimpleAverage

    # Uncertainty Forest model is used as transformer
    elif model == "uf":

        train_x1 = train_x1.reshape((train_x1.shape[0], train_x1.shape[1] * train_x1.shape[2] * train_x1.shape[3]))
        tmp_data = tmp_data.reshape((tmp_data.shape[0], tmp_data.shape[1] * tmp_data.shape[2] * tmp_data.shape[3]))
        test_x = test_x.reshape((test_x.shape[0], test_x.shape[1] * test_x.shape[2] * test_x.shape[3]))

        default_transformer_class = TreeClassificationTransformer
        # Max depth of tree is set
        default_transformer_kwargs = {"kwargs" : {"max_depth" : 5}}

        default_voter_class = TreeClassificationVoter
        default_voter_kwargs = {}

        default_decider_class = SimpleAverage
        
    return (train_x1, test_x, tmp_data, default_transformer_class, default_transformer_kwargs, 
            default_voter_class, default_voter_kwargs, default_decider_class)

In [ ]:
# Runs the experiments
def LF_experiment(data_x, data_y, angle, model, granularity, reps=1, ntrees=29, acorn=None):
    if acorn is not None:
        np.random.seed(acorn)
    
    errors = np.zeros(2)
    
    with tf.device('/gpu:'+str(int(angle //  granularity) % 4)):
        for rep in range(reps):
            print("rep:{}".format(rep))
            train_x1, train_y1, train_x2, train_y2, test_x, test_y = cross_val_data(data_x, data_y, total_cls=10)

            # change data angle for second task
            tmp_data = train_x2.copy()
            _tmp_ = np.zeros((32,32,3), dtype=int)
            total_data = tmp_data.shape[0]

            for i in range(total_data):
                tmp_ = image_aug(tmp_data[i],angle)
                tmp_data[i] = tmp_
                
            # Call choose_transformer function
            (train_x1, test_x, tmp_data, default_transformer_class, default_transformer_kwargs, 
             default_voter_class, default_voter_kwargs, default_decider_class) = choose_transformer(train_x1, test_x, 
                                                                                                    test_y, tmp_data)
            
            if model == "uf":
                progressive_learner = LifelongClassificationForest(default_transformer_class = default_transformer_class, 
                                                         default_transformer_kwargs = default_transformer_kwargs,
                                                         default_voter_class = default_voter_class,
                                                         default_voter_kwargs = default_voter_kwargs,
                                                         default_decider_class = default_decider_class)
            elif model == "dnn":
                progressive_learner = LifelongClassificationNetwork(default_transformer_class = default_transformer_class, 
                                                         default_transformer_kwargs = default_transformer_kwargs,
                                                         default_voter_class = default_voter_class,
                                                         default_voter_kwargs = default_voter_kwargs,
                                                         default_decider_class = default_decider_class)

            progressive_learner.add_task(
                X = train_x1, 
                y = train_y1,
                # 67% of the data is used for transformers, 33% is used for voters
                transformer_voter_decider_split = [0.67, 0.33, 0],
                decider_kwargs = {"classes" : np.unique(train_y1)}
            )

            progressive_learner.add_transformer(
                X = tmp_data, 
                y = train_y2,
                transformer_data_proportion = 1,
                backward_task_ids = [0]
            )

            llf_task1=progressive_learner.predict(test_x, task_id=0)
            llf_single_task=progressive_learner.predict(test_x, task_id=0, transformer_ids=[0])

            errors[1] = errors[1]+(1 - np.mean(llf_task1 == test_y))
            errors[0] = errors[0]+(1 - np.mean(llf_single_task == test_y))
    
    errors = errors/reps
    print("Errors For Angle {}: {}".format(angle, errors))
    
    return(errors)

In [ ]:
# Rotates the image by the given angle and zooms in to remove unnecessary white space at the corners
def image_aug(pic, angle, centroid_x=23, centroid_y=23, win=16, scale=1.45):
    im_sz = int(np.floor(pic.shape[0]*scale))
    pic_ = np.uint8(np.zeros((im_sz,im_sz,3),dtype=int))
    
    pic_[:,:,0] = ndimage.zoom(pic[:,:,0],scale)
    
    pic_[:,:,1] = ndimage.zoom(pic[:,:,1],scale)
    pic_[:,:,2] = ndimage.zoom(pic[:,:,2],scale)
    
    image_aug = rotate(pic_, angle, resize=False)
    #print(image_aug.shape)
    image_aug_ = image_aug[centroid_x-win:centroid_x+win,centroid_y-win:centroid_y+win,:]
    
    return img_as_ubyte(image_aug_)

# Hyperparameters

Hyperparameters determine how the model will run. Changing the value of `model` to `"uf"` will run the L2F algorithm, while `"dnn"` will run the L2N algorithm.

`granularity` refers to the amount by which the angle will be increased each time. Setting this value at 1 will cause the algorithm to test every whole number rotation angle between 0 and 180 degrees.

`reps` refers to the number of repetitions tested for each angle of rotation. For each repetition, the data is randomly resampled/

In [ ]:
### MAIN HYPERPARAMS ###
model = "uf"
granularity = 45
reps = 5
########################

In [ ]:
# Loads and reshapes data sets
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar100.load_data()
data_x = np.concatenate([X_train, X_test])
data_y = np.concatenate([y_train, y_test])
data_y = data_y[:, 0]

In [ ]:
# Runs the experiment at a new angle of rotation
def perform_angle(angle):
    error_list = LF_experiment(data_x, data_y, angle, model, granularity, reps=reps, ntrees=16, acorn=1)
    
    return(error_list)

In [ ]:
# Run L2N
if model == "dnn":
    for angle_adder in range(0, 181, granularity * 4):
        angles = angle_adder + np.arange(0, granularity * 4, granularity)
        with Pool(4) as p:
            errors_array = p.map(perform_angle, angles)
            
# Run L2F
elif model == "uf":
    angles = np.arange(0, 181, granularity)
    with Pool(8) as p:
        errors_array.append(p.map(perform_angle, angles))

# Rotation CIFAR Plot

This section takes the results of the experiment and plots the backward transfer efficiency against the angle of rotation for the images in **CIFAR-100**.

In [ ]:
#Choose which algorithms to plot
alg_name = ['L2F']#['L2N','L2F','LwF','EWC','Online_EWC','SI']
angles = np.arange(0,181,45)
tes = [[] for _ in range(len(alg_name))]

for algo_no,alg in enumerate(alg_name):
    for angle in angles:
        orig_error, transfer_error = errors_array[int(angle/granularity)]
        tes[algo_no].append(orig_error / transfer_error)

In [ ]:
#Choose which color to make each algorithm's results
clr = ["#00008B"]#["#00008B", "#e41a1c", "#4daf4a", "#984ea3", "#ff7f00", "#CCCC00"]
c = sns.color_palette(clr, n_colors=len(clr))
fig, ax = plt.subplots(1,1, figsize=(8,8))

#Plot the data
for alg_no,alg in enumerate(alg_name):
    if alg_no<2:
        ax.plot(angles,tes[alg_no], c=c[alg_no], label=alg_name[alg_no], linewidth=3)
    else:
        ax.plot(angles,tes[alg_no], c=c[alg_no], label=alg_name[alg_no])

#ax.set_yticks([.9,.95, 1, 1.05,1.11])
#ax.set_ylim([0.85,1.13])
ax.set_xticks([0,30,60,90,120,150,180])
ax.tick_params(labelsize=20)
ax.set_xlabel('Angle of Rotation (Degrees)', fontsize=24)
ax.set_ylabel('Backward Transfer Efficiency', fontsize=24)
ax.set_title("Rotation Experiment", fontsize = 24)
right_side = ax.spines["right"]
right_side.set_visible(False)
top_side = ax.spines["top"]
top_side.set_visible(False)
plt.tight_layout()
#x.legend(fontsize = 24)
plt.show()

# FAQs

### Why am I getting an "out of memory" error?
`Pool(8)` in the previous cell allows for parallel processing, so the number within the parenthesis should be, at max, the number of cores in the device on which this notebook is being run. Even if a warning is produced, the results of the experimented should not be affected.

### Why is this taking so long to run? How can I speed it up to see if I am getting the expected outputs?
Changing the maximum tree depth does not affect the runtime much. Decreasing the value of `reps`, decreasing the value of `num_transformers`, or increasing the value of `granularity` will all decrease runtime at the cost of noisier results.